In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
df_p = pd.read_csv('./Data/Pedestrian_Counting_System_-_Monthly__counts_per_hour_.csv')

In [3]:
df_s = pd.read_csv('./Data/Pedestrian_Counting_System_-_Sensor_Locations.csv')

In [4]:
df_s_location = df_s[['sensor_id', 'latitude', 'longitude']]

In [5]:
map_loc_heat = folium.Map(location=[-37.8140000, 144.9633200],tiles = "Stamen Toner",
                    zoom_start = 14) 

# Ensure you're handing it floats
df_s['latitude'] = df_s['latitude'].astype(float)
df_s['longitude'] = df_s['longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df = df_s[['latitude', 'longitude']]

# List comprehension to make out list of lists
heat_data = [[row['latitude'],row['longitude']] for index, row in heat_df.iterrows()]

# Styling title
loc = 'Corpus Christi'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>Locations of Sensors</b></h3>
             '''.format(loc) 
map_loc_heat.get_root().html.add_child(folium.Element(title_html))

# Plot it on the map
HeatMap(heat_data, blur=10, radius=15).add_to(map_loc_heat)

# Display the map
map_loc_heat

In [6]:
map_loc = folium.Map(location=[-37.8140000, 144.9633200],tiles = "Stamen Toner",
                    zoom_start = 14) 

# Ensure you're handing it floats
df_s['latitude'] = df_s['latitude'].astype(float)
df_s['longitude'] = df_s['longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
#heat_df = df_s[['latitude', 'longitude']]

# List comprehension to make out list of lists
#heat_data = [[row['latitude'],row['longitude']] for index, row in heat_df.iterrows()]

for index, row in df_s.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['sensor_description']).add_to(map_loc)

# Styling title
loc = 'Corpus Christi'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>Locations of Sensors</b></h3>
             '''.format(loc) 
map_loc.get_root().html.add_child(folium.Element(title_html))

# Plot it on the map
#HeatMap(heat_data, blur=10, radius=15).add_to(map_hooray)

# Display the map
map_loc

These heatmaps above showing the locations of the sensors. We choose the same zoom which we are using later in the heatmaps showing the amount of pedestrians in specific months. As you may see, one of the sensors is not really visible with this zoom. We are aware of that but since it wasn't active in any of the two months we choose for the later maps, we decided to focus the observers attention on the center of the city.

In [7]:
# looking at the sensor located futher outside the city
df_s.loc[df_s['sensor_description'] == 'Macaulay Rd- Bellair St']

,sensor_id,sensor_description,sensor_name,installation_date,status,note,direction_1,direction_2,latitude,longitude,location
71,76,Macaulay Rd- Bellair St,KenMac_T,2021/02/20,A,NaN,West,East,-37.794538,144.930362,"(-37.79453802, 144.93036194)"


Now, we can see that the sensor was installed during covid in the year 2021. For the later maps, we choose to visualize months of a year which is not biased by covid (2019) and therefore the sensor is not visible on these maps.

In [8]:
# getting data frames from January and July in 2019
df_p_19 = df_p.loc[df_p['Year'] == 2019]
df_p_19_ja = df_p_19.loc[df_p_19['Month'] == 'January']
df_p_19_ju = df_p_19.loc[df_p_19['Month'] == 'July']

In [9]:
# merging the january dataset with the sensors location
df_p_19_ja_g = df_p_19_ja.groupby(['Sensor_ID']).sum()
df_p_19_ja_g.columns = df_p_19_ja_g.columns.str.replace('Hourly_Counts', 'Monthly_Counts')
df_p_19_ja_g = pd.merge(df_p_19_ja_g, df_s_location, left_on="Sensor_ID", right_on="sensor_id").drop('sensor_id', axis=1)
# normalizing the monthly count such that we can easier compute the heat map
MC_sum_ja = df_p_19_ja_g['Monthly_Counts'].sum()
df_p_19_ja_g.Monthly_Counts = (df_p_19_ja_g.Monthly_Counts / MC_sum_ja) * 10000

# merging the july dataset with the sensors location
df_p_19_ju_g = df_p_19_ju.groupby(['Sensor_ID']).sum()
df_p_19_ju_g.columns = df_p_19_ju_g.columns.str.replace('Hourly_Counts', 'Monthly_Counts')
df_p_19_ju_g = pd.merge(df_p_19_ju_g, df_s_location, left_on="Sensor_ID", right_on="sensor_id").drop('sensor_id', axis=1)
# normalizing the monthly count such that we can easier compute the heat map
MC_sum_ju = df_p_19_ju_g['Monthly_Counts'].sum()
df_p_19_ju_g.Monthly_Counts = (df_p_19_ju_g.Monthly_Counts / MC_sum_ju) * 10000

In [10]:
# creating the heat map for january
map_january = folium.Map(location=[-37.8140000, 144.9633200],tiles = "Stamen Toner",
                    zoom_start = 14) 

# Ensure you're handing it floats
df_p_19_ja_g['latitude'] = df_p_19_ja_g['latitude'].astype(float)
df_p_19_ja_g['longitude'] = df_p_19_ja_g['longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df_ja = pd.DataFrame(columns=['latitude',
                    'longitude'])
for index, row in df_p_19_ja_g.iterrows():
    for index in range(0, int(row['Monthly_Counts'])):
        df_row = pd.DataFrame([[row['latitude'],row['longitude']]], columns=['latitude', 'longitude'])
        heat_df_ja = heat_df_ja.append(df_row, ignore_index = True)

# List comprehension to make out list of lists
heat_data_ja = [[row['latitude'],row['longitude']] for index, row in heat_df_ja.iterrows()]

# Styling
loc = 'Corpus Christi'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>Normalized Amount of Pedestrians in January</b></h3>
             '''.format(loc) 
map_january.get_root().html.add_child(folium.Element(title_html))

# Plot it on the map
HeatMap(heat_data_ja, blur=10, radius=15).add_to(map_january)

# Display the map
map_january

In [11]:
# creating the heat map for july
map_july = folium.Map(location=[-37.8140000, 144.9633200],tiles = "Stamen Toner",
                    zoom_start = 14) 

# Ensure you're handing it floats
df_p_19_ju_g['latitude'] = df_p_19_ju_g['latitude'].astype(float)
df_p_19_ju_g['longitude'] = df_p_19_ju_g['longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df_ju = pd.DataFrame(columns=['latitude',
                    'longitude'])
for index, row in df_p_19_ju_g.iterrows():
    for index in range(0, int(row['Monthly_Counts'])):
        df_row = pd.DataFrame([[row['latitude'],row['longitude']]], columns=['latitude', 'longitude'])
        heat_df_ju = heat_df_ju.append(df_row, ignore_index = True)

# List comprehension to make out list of lists
heat_data_ju = [[row['latitude'],row['longitude']] for index, row in heat_df_ju.iterrows()]

# Styling
loc = 'Corpus Christi'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>Normalized Amount of Pedestrians in July</b></h3>
             '''.format(loc) 
map_july.get_root().html.add_child(folium.Element(title_html))

# Plot it on the map
HeatMap(heat_data_ju, blur=10, radius=15).add_to(map_july)

# Display the map
map_july